In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
chrome_path = r'C:\Users\dlagm\workspace_kdt\lecture\selenium\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# driver = webdriver.Chrome(
# service=Service(ChromeDriverManager().install()) )
driver = webdriver.Chrome()

C:\Users\dlagm\AppData\Local\Temp\ipykernel_14244\268815354.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


In [8]:
!pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [9]:
import pandas as pd

# Excel 파일 읽기
df = pd.read_excel('./category.xls', engine='xlrd')

# 데이터 확인
print(df.head())


     카테고리번호      대분류       중분류    소분류          세분류
0  50003307  가구/인테리어  DIY자재/용품  가구부속품         가구다리
1  50003308  가구/인테리어  DIY자재/용품  가구부속품         가구바퀴
2  50003309  가구/인테리어  DIY자재/용품  가구부속품  경첩/꺽쇠/자석철물류
3  50003314  가구/인테리어  DIY자재/용품  가구부속품      기타가구부속품
4  50003310  가구/인테리어  DIY자재/용품  가구부속품         나사/못


In [14]:
import time
URL = 'https://search.shopping.naver.com/search/category/?catId={}&origQuery&pagingIndex={}&productSet=model'.format(50003307,2)


print(URL)

https://search.shopping.naver.com/search/category/?catId=50003307&origQuery&pagingIndex=2&productSet=model


In [15]:
cate_num = []
cat1 = []
cat2 = []
cat3 = []
for idx, row in df.iterrows():
    cate_num.append(row['카테고리번호'])
    cat1.append(row['대분류'])
    cat2.append(row['중분류'])
    cat3.append(row['소분류'])

In [17]:
len(cate_num)

4818

In [35]:
for num in cate_num:
    
    URL_1 = 'https://search.shopping.naver.com/search/category/?catId={}'.format(num) # 대분류

    
    for page in range(1,11): # 한 페이지 40개씩 총 400개
        URL = URL_1 + '&origQuery&pagingIndex={}&productSet=model'.format(page)
        
        print(URL)
        driver.get(URL)

        # 스크롤 끝까지 내리기
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)


        names = []
        image_list = []
        price = []
        price_list = []
        brand = []
        cat_1 = []
        cat_2 = []
        cat_3= []

        while True:
            
            items_list = driver.find_elements(By.CSS_SELECTOR, '.basicList_list_basis__uNBZx .basicList_item__0T9JD')
            print(len(items_list))
            
            for item in items_list: # 하나의 카테고리 안에서 상품 가져오기

                # 카테고리 대중소
                categories = item.find_elements(By.CLASS_NAME, 'basicList_category__cXUaZ')
                cate1 = categories[0].text
                cate2 = categories[1].text
                cate3 = categories[2].text
                cat_1.append(cate1)
                cat_2.append(cate2)
                cat_3.append(cate3)   

                # 제품 이름 
                try: 
                    product_name = item.find_element(By.CLASS_NAME, 'basicList_title__VfX3c').text
                except:
                    product_name = item.find_element(By.CLASS_NAME, 'linkAnchor').text
                names.append(product_name)

                print(product_name)
                

                # 가격
                try:
                    price_one = item.find_element(By.CLASS_NAME, 'price_num__S2p_v').text
                except:
                    element = item.find_element(By.XPATH, './/*[@id="content"]/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/strong/span/span/span[2]')
                    price_one = element.text

                price.append(price_one)

                # 브랜드
                brand_clk = item.find_element(By.CSS_SELECTOR, '.thumbnail_thumb_wrap__RbcYO') # 자세한 링크로 들어가기
                brand_clk.click()
                time.sleep(3)

                driver.switch_to.window(driver.window_handles[-1]) # 새 창으로 드라이브 이동
                driver_new = driver

                brand_one = driver.find_elements(By.CSS_SELECTOR, '.top_info_inner__aM_0Z .top_cell__5KJK9 em')
                brand_name = brand_one[1].text
                brand.append(brand_name)


                # 이미지 링크
                image_element = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[1]/div/div/img')
                image_src = image_element.get_attribute('src')

                image_list.append(image_src)

                # 최저가 리스트
                low_list = driver_new.find_elements(By.CSS_SELECTOR, '.productList_seller_wrap__FZtUS li')
                price_low_list = []
                for li in low_list:
                    manyofone = li.find_element(By.CSS_SELECTOR, '.productList_price__2eGt4 em').text
                    price_low_list.append(manyofone)
                price_list.append(price_low_list)


                driver_new.close() # 뒤로가기
                driver.switch_to.window(driver.window_handles[0]) # 드라이브 전환
                
            if item == items_list[-1]:
                break

https://search.shopping.naver.com/search/category/?catId=50003307&origQuery&pagingIndex=1&productSet=model
40
곽씨네슈퍼 가구 높이조절 의자 테이블 책상 높이조절발 다리 높이기 침대 받침 연장
아키아 테이블 원목다리 책상 나무기둥 고하중 60T 철물보강형
아크릴 가구 다리 받침 매트 바닥 보호 식탁 쇼파 다리 받침대 층간소음 눌림 패임 방지
타공 링 아크릴 가구 다리 받침 식탁 쇼파 바닥 매트
접이식상다리 철좌탁다리 2개입
원목다리 테이블 나무 상다리 캠핑테이블다리 520 네츄럴
책상다리 가구 테이블 철재 다리 키높이 높이조절다리 바퀴 상다리 이반가구11
테이블 책상 다리 식탁 가구 철제 우드슬랩 좌탁 철재 바테이블 헤어핀 3칼라
H형접이식 잘 부러지지 않는 강화플라스틱 상다리 양발다리 2P 테이블1벌
곽씨네슈퍼 책상 높이조절 가구 다리 높이조절발 테이블 의자 식탁 높이기 연장 받침 원형
pvc 폴딩다리 폴딩박스 사선 다리 발 분리형
철물박사 PVC 높이 조절발 고무발 가구 책상다리용
모네스페스 철제 가구 테이블 다리 책상 식탁 좌탁 벤치 프레임 우드슬랩다리
남원공방 상다리부속 밥상다리 교자상다리부속
국산 접이식 상다리 밥상다리 부속품 책상
업소용 VFC-도도 탁자 목재다리의자 식탁
파미가구 스트롱 테이블다리 접이식 프레임 책상
조절발 책상 의자 수평 높이 조절 가구 다리 높이 사각 파이프캡 BH
접이식 상다리 밥상 식탁 다리 책상 테이블 높이조절
워싱턴 스텐 H형 테이블 다리 HFL-3352S
우드원형 책상 식탁 다리 높이조절발 4P
상다리부속 접이식상다리
펜션 테이블 디자인 원형골드 스텐다리 GP-ITL10
가구다리 플라스틱 철제 원목 서랍장 거실장 높이조절 상다리 270
상다리 PVC 접이식상다리 cb 택색상
우드슬랩다리 철제다리 테이블다리 사다리형
사선형 원목 책상 식탁 테이블 철제다리 600
가구다리 식탁 접이식 테이블 상다리 골발 높이조절 폴딩 철제 6 나팔발무광 

KeyboardInterrupt: 

In [21]:
names

['스틸 테이블 다리 700 CZ4310',
 '해외일자 나무 목재 가구 책상 테이블 높이조절발 A',
 '스틸 테이블 다리 700 CZ4310',
 '해외일자 나무 목재 가구 책상 테이블 높이조절발 A',
 '스틸 테이블 다리 700 CZ4310']

In [31]:
items_list

[<selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2386")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2387")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2388")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2389")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2390")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6acb0d952d53d29df0f", element="BFC50D03160DE3D22D2B2C612D988727_element_2391")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2385b0bf26d2c6ac